In [6]:
import pandas as pd

In [7]:
aqi_df = []
aqi_files_start_year = 2017

for i in range(5) :
    # Load the data into a dataframe
    df = pd.read_csv('raw_data/annual_aqi_by_county_' + str(i + aqi_files_start_year) + '.csv')
    #print(df.info())

    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df[df['state'] == 'California'].drop(columns=['state'])
    
    aqi_df.append(df)

In [ ]:
# combine all aqi years dataframes into one
combined_aqi_df = pd.concat(aqi_df, ignore_index=True)
#print(combined_aqi_df)

In [ ]:
combined_aqi_df.to_csv('processed_data/combined_aqi_data.csv')

In [10]:
# clean asthma emergency department visits data

asthma_df = []
asthma_files_start_year = 2017

for i in range(5) :
    # load the data into a dataframe
    df = pd.read_excel('raw_data/Asthma_Emergency_' + str(i + asthma_files_start_year) + '.xlsx')
    #print(df.info())

    # clean up the dataframe
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df.rename(columns={'counties': 'county'}, inplace=True)
    df = df[df['county'] != 'California'] # remove rows with 'California' in county name
    df = df.drop(columns=['lower_95%_limit', 'upper_95%_limit'])  # optional, but cleaner
 
    # add year column and reorder columns to have year after county (like in aqi_df)
    df['year'] = i + asthma_files_start_year
    columns_order = ['county', 'year'] + [col for col in df.columns if col not in ['county', 'year']]
    df = df[columns_order]

    asthma_df.append(df)


In [11]:
# combine all asthma years dataframes into one
combined_asthma_df = pd.concat(asthma_df, ignore_index=True)
#print(combined_asthma_df)

        county  year  age-adjusted_rate_per_10,000  number_of_cases
0      Alameda  2017                         56.04           8916.0
1       Alpine  2017                           NaN              NaN
2       Amador  2017                         56.02            175.0
3        Butte  2017                         45.29            935.0
4    Calaveras  2017                         54.51            197.0
..         ...   ...                           ...              ...
285     Tulare  2021                         26.95           1280.0
286   Tuolumne  2021                         23.59            104.0
287    Ventura  2021                         16.81           1312.0
288       Yolo  2021                         22.87            436.0
289       Yuba  2021                         29.34            235.0

[290 rows x 4 columns]


In [12]:
combined_asthma_df.to_csv('processed_data/combined_asthma_data.csv')

In [ ]:
# how many distinct years each county appears  (check if all listed counties have data for all years)
year_counts = combined_aqi_df.groupby('county')['year'].nunique()
print(year_counts.sort_values())